# 1. 데이터 로드

In [1]:
import os

import sys
import time
import random
import datetime
import requests
import pandas as pd
import numpy as np
import hashlib, hmac, base64
from itertools import combinations, permutations
from dtw import *
import json
import urllib.request
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
import pickle
from pytz import timezone
from difflib import SequenceMatcher

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict
from pytrends.request import TrendReq
import models.google.google_trend as google_trend
import nest_asyncio
import asyncio

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# 2. API설정

In [2]:
from api_set import APIClient

# API 설정
from utils import get_secret
BASE_URL = get_secret("BASE_URL")
CUSTOMER_ID = get_secret("CUSTOMER_ID")
API_KEY = get_secret("API_KEY")
SECRET_KEY = get_secret("SECRET_KEY")
URI = get_secret("URI")
METHOD = get_secret("METHOD")
# API 클라이언트 인스턴스 생성
api_client = APIClient(BASE_URL, CUSTOMER_ID, API_KEY, SECRET_KEY,URI,METHOD)


# 3. 연관검색어 수집

In [3]:
# 키 로드
from utils import load_keywords 
keywords_data = load_keywords('main_keyword.json')

from utils import get_today_date
# 오늘의 날짜 가져오기
formatted_today, day = get_today_date()


# 결과 저장 폴더 생성
from utils import make_directory

make_directory('./data')
make_directory('./data/rl_srch')
make_directory(f'./data/rl_srch/{day}')  # 키워드별 연관검색어 리스트 저장

In [4]:

# 검색어 리스트와 결과 저장 경로 설정
srch_keyword = ['keyword_final']  
save_path = './data/rl_srch/'  
print(api_client.base_url)

https://api.searchad.naver.com


In [5]:
srch_keyword

['keyword_final']

In [6]:
import os
import csv
import datetime
import asyncio
import pandas as pd

# 필요한 경우 비동기를 위한 nest_asyncio 적용
import nest_asyncio
nest_asyncio.apply()

from collect_keywords import collect_keywords

async def main(srch_keyword, day):
    # 오늘 날짜로 폴더 경로 생성
    folder_path = './data/rl_srch/' + datetime.datetime.now().strftime('%y%m%d')
    file_path = f"{folder_path}/collected_keywords.csv"
    
    # 폴더가 존재하는지 확인
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    # 파일이 존재하는지 확인
    if os.path.isfile(file_path):
        # 파일이 존재하면, 데이터를 읽어옵니다.
        collected_keywords_data = pd.read_csv(file_path)
    else:
        # 파일이 없으면, collect_keywords 함수를 호출해서 데이터를 수집합니다.
        collected_keywords_data = await collect_keywords(srch_keyword, day)
        # 결과를 CSV로 저장
        collected_keywords_data.to_csv(file_path, index=False)
    
    return collected_keywords_data
collected_keywords_data=asyncio.run(main(srch_keyword, day))


In [7]:

collected_keywords_dat_copy=asyncio.run(main(srch_keyword, day))

In [8]:
from utils import merge_and_mark_duplicates_limited
# '검색어'별로 그룹화된 DataFrame을 리스트에 저장
df_list = [group for _, group in collected_keywords_data.groupby('검색어')]

collected_keywords_data = merge_and_mark_duplicates_limited(df_list)


collected_keywords_data.head(30)

,연관키워드,월간검색수_합계,검색어,중복검색어
0,CMA금리비교,29620.0,CMA금리비교,"CMA금리비교,CMA통장,ELS,금리"
1,파킹통장금리비교,57100.0,CMA금리비교,"CMA금리비교,CMA통장,금리"
2,CMA통장,55600.0,CMA금리비교,"CMA금리비교,CMA통장,금리,신규상장,채권"
3,CMA,147000.0,CMA금리비교,"CMA금리비교,CMA통장,금리"
4,KB증권CMA,3010.0,CMA금리비교,"CMA금리비교,CMA통장,금리"
5,정기예금금리비교,136600.0,CMA금리비교,"CMA금리비교,CMA통장"
6,파킹통장,401900.0,CMA금리비교,"CMA금리비교,CMA통장,금리"
7,예금금리,191400.0,CMA금리비교,"CMA금리비교,CMA통장,금융상품"
8,예금이자높은은행,122500.0,CMA금리비교,"CMA금리비교,CMA통장"
9,적금이자높은은행,121200.0,CMA금리비교,"CMA금리비교,CMA통장"


In [9]:
import utils

def add_client_info(collected_keywords_data, start_id_index=1):
    clients = utils.get_secret("clients")
    start_id_index = 1
    clients = utils.get_secret("clients")
    # ID와 PW 컬럼을 데이터프레임에 추가하는 로직
    total_rows = len(collected_keywords_data)
    ids = []
    pws = []

    for i in range(total_rows):
        # 현재 id 인덱스 계산 (start_id_index를 기준으로)
        current_id_index = ((i // 500) + start_id_index) % len(clients)
        current_id_key = f"id_{current_id_index}"
        
        # 현재 id와 pw 할당
        current_id = clients[current_id_key]['client_id']
        current_pw = clients[current_id_key]['client_secret']
        
        ids.append(current_id)
        pws.append(current_pw)

    # ID와 PW 컬럼 추가
    collected_keywords_data['id'] = ids
    collected_keywords_data['pw'] = pws

    return collected_keywords_data
collected_keywords_data= add_client_info(collected_keywords_data)
new_columns = ['일별급상승', '주별급상승', '월별급상승', '주별지속상승', '월별지속상승', '월별규칙성']

for column in new_columns:
    collected_keywords_data[column] = 0
collected_keywords_data.head(5)

,연관키워드,월간검색수_합계,검색어,중복검색어,id,pw,일별급상승,주별급상승,월별급상승,주별지속상승,월별지속상승,월별규칙성
0,CMA금리비교,29620.0,CMA금리비교,"CMA금리비교,CMA통장,ELS,금리",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od,0,0,0,0,0,0
1,파킹통장금리비교,57100.0,CMA금리비교,"CMA금리비교,CMA통장,금리",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od,0,0,0,0,0,0
2,CMA통장,55600.0,CMA금리비교,"CMA금리비교,CMA통장,금리,신규상장,채권",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od,0,0,0,0,0,0
3,CMA,147000.0,CMA금리비교,"CMA금리비교,CMA통장,금리",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od,0,0,0,0,0,0
4,KB증권CMA,3010.0,CMA금리비교,"CMA금리비교,CMA통장,금리",eywsrQuPMFtlbYz1XAl7,Me4ra4s8od,0,0,0,0,0,0


In [10]:
def groupped_df(name,collected_keywords_data):
    grouped = collected_keywords_data.groupby(name)
    df_list = [group for _, group in grouped]
    return df_list
df_list=groupped_df('id',collected_keywords_data)
print(len(df_list))

3


In [11]:
import asyncio
import trend 
# 비동기 메인 함수 수정
async def trend_main(df, clients):
    # 파라미터 설정
    params = {
        "search_keywords": list(df['연관키워드']),
        "id": df['id'].iloc[0],
        "pw": df['pw'].iloc[0],
        "api_url": "https://openapi.naver.com/v1/datalab/search",
        "name": '연관검색어'
    }
    api_url = "https://openapi.naver.com/v1/datalab/search"
    
    # trend_maincode 함수 실행
    results = await trend.trend_maincode(params, clients, api_url)
    return results

async def run_all(df_list, clients):
    tasks = [trend_main(df, clients) for df in df_list]
    results = await asyncio.gather(*tasks)
    return results

clients = get_secret("clients")  # clients 정보를 로드
# 이벤트 루프 실행

trend_main_data = asyncio.run(run_all(df_list, clients))
results =trend_main_data.copy()

In [12]:
from select_keyword import select_keyword, rising_keyword_analysis, monthly_rule

start_time = time.time()
select_periods = ['daily', 'weekly', 'month']
rising_periods=['weekly', 'month']

formatted_today, today_date = utils.get_today_date()
month_rule_list=[]
select_list=[[],[],[]]

rising_list=[[],[]]
rising_month_list=[]
# 월별, 주별, 일별 키워드 분석 실행

    # 각 분석 기간에 대해 결과 집합을 순회합니다.
for keyword_group in results:
    # 키워드 그룹의 각 키워드 데이터프레임에 대해 순회합니다.
    for keyword_data in keyword_group:
        # 월별 규칙을 적용하여 결과를 가져옵니다.
        monthly_data, monthly_chart, similarity_rate, rising_months = monthly_rule(keyword_data, today_date, 'month')
        
        if monthly_data is not None:
            # 결과 데이터프레임의 열 이름을 가져옵니다.
            column_names = monthly_data.columns
            rising_month_list.append([rising_months,column_names[0]])
            # 결과 데이터프레임에서 값 리스트를 추출합니다.
            data_values_list = monthly_data[column_names].values
            # 월별 차트에 데이터 값을 추가합니다.
            monthly_chart['Indicator'] = data_values_list
            monthly_chart['InfoData'] = similarity_rate
            # 상승 월 정보를 추가합니다. 상승 월이 없는 경우 0으로 설정합니다.
            monthly_chart['RisingMonth'] = 0
            
            # 최종 결과 리스트에 수정된 월별 차트를 추가합니다.
            month_rule_list.append(monthly_chart)
                
# 주별, 월별 상승 키워드 분석 실행
rising_analysis_periods = ['weekly', 'month']
i=0
for period in rising_analysis_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            rising_tmp, rising_graph, rising_info = rising_keyword_analysis(keyword_df, today_date, period)
            if rising_tmp is not None:
                column_names=rising_tmp.columns
                data_values_list = rising_tmp[column_names].values
                rising_graph['Indicator'] = data_values_list
                rising_graph['InfoData'] = rising_info

                rising_list[i].append(rising_graph)
    i=i+1

    


i = 0
# 일별, 주별, 월별 키워드 선택 실행
for period in select_periods:
    for keyword_df_group in results:
        for keyword_df in keyword_df_group:
            selected_tmp, selected_graph, selected_info = select_keyword(keyword_df, today_date, period)
            if selected_graph is not None:
                # 데이터프레임의 열 이름을 출력합니다.
                selected_graph['InfoData'] = selected_info
                select_list[i].append(selected_graph)
            else:
                pass
    i += 1

end_time = time.time()
print(f"Analysis completed in {end_time - start_time} seconds.")

월별 규칙성 키워드 발견 : 아파트월세
[1, 2, 3, 4, 7, 10, 12]
월별 규칙성 키워드 발견 : 자동차보험
[1, 2, 3, 5, 6, 7, 10, 12]
월별 규칙성 키워드 발견 : 소득세계산법
[1, 2, 3, 5, 7, 10, 11, 12]
월별 규칙성 키워드 발견 : 공모전
[2, 3, 5, 7, 10]
월별 규칙성 키워드 발견 : 증여세율
[1, 2, 3, 4, 5, 7, 10, 11]
월별 규칙성 키워드 발견 : 채권추심
[1, 2, 3, 5, 6, 7, 10]
월별 규칙성 키워드 발견 : 차용증법적효력
[2, 3, 7, 10, 12]
월별 규칙성 키워드 발견 : 약정서
[1, 2, 3, 5, 6, 7, 10, 11]
월별 규칙성 키워드 발견 : 차용
[1, 2, 3, 4, 6, 7, 10, 11]
월별 규칙성 키워드 발견 : 신용카드발급
[1, 2, 3, 5, 7, 10, 11]
월별 규칙성 키워드 발견 : 노무사상담
[1, 2, 3, 5, 7, 10, 11]
월별 규칙성 키워드 발견 : 노후준비
[2, 3, 5, 7, 10, 11, 12]
월별 규칙성 키워드 발견 : 노무법인
[1, 2, 3, 7, 10, 11]
월별 규칙성 키워드 발견 : 경제잡지
[2, 3, 5, 7, 10, 11, 12]
월별 규칙성 키워드 발견 : 투자종류
[2, 3, 5, 7, 10, 11, 12]
월별 규칙성 키워드 발견 : 돈버는앱
[1, 2, 3, 5, 7, 9, 12]
월별 규칙성 키워드 발견 : 수요조사
[1, 2, 3, 7, 10]
월별 규칙성 키워드 발견 : 알바
[2, 3, 6, 7, 10, 11]
월별 규칙성 키워드 발견 : 월세
[1, 2, 3, 4, 7, 10, 12]
월별 규칙성 키워드 발견 : 상가임대
[1, 2, 3, 5, 7, 10]
월별 규칙성 키워드 발견 : 부동산명함
[2, 3, 7, 10, 11]
월별 규칙성 키워드 발견 : 명함제작
[1, 2, 3, 7, 10, 12]
월별 규칙성 키워드 발견 : 월세방
[2, 3, 4,

# Graph_result

In [47]:
import pandas as pd

# 리스트와 유형을 매핑
lists_and_types = [
    (select_list[0], '일별급상승'),
    (select_list[1], '주별급상승'),
    (select_list[2], '월별급상승'),
    (rising_list[0], '주별지속상승'),
    (rising_list[1], '월별지속상승'),
    (month_rule_list, '월별규칙성')
]

# 각 리스트에 대한 유형 라벨 추가 및 병합
def process_and_concat(df_list, label):
    for df in df_list:
        df['유형'] = label
    return pd.concat(df_list).reset_index(drop=True)

# 모든 리스트를 처리하고 하나의 데이터프레임으로 병합
processed_dfs = [process_and_concat(df_list, label) for df_list, label in lists_and_types]

graph_result = pd.concat(processed_dfs).reset_index(drop=True)

# 불필요한 컬럼 삭제 및 '주간지속상승'을 '주별지속상승'으로 수정
graph_result = graph_result.drop(columns=['InfoData', 'Indicator', 'RisingMonth'])
graph_result['유형'].replace({'주간지속상승': '주별지속상승'}, inplace=True)

# 정렬
graph_result.sort_values(by=['연관검색어', '유형', '검색일자'], ascending=[True, True, True], inplace=True)

# 최종 결과 출력
graph_result.reset_index(drop=True, inplace=True)


In [16]:
flags_and_lists = [
    ("일별 급상승", select_list[0]),
    ("주별 급상승", select_list[1]),
    ("주별 지속상승", rising_list[0]),
    ("월별 급상승", select_list[2]),
    ("월별 지속상승", rising_list[1]),
    ("월별 규칙성", month_rule_list),
]

for flag_name, data_list in flags_and_lists:
    utils.update_keywords_flag(collected_keywords_data, data_list, flag_name)


In [294]:
 # process_data : 지정된 조건에 따라 데이터를 필터링하고, 추가 처리를 통해 최종 데이터프레임을 반환하는 함수.

info_result_daily_select = utils.process_data(collected_keywords_data, '일별 급상승', '일별 급상승', select_list[0])
info_result_weekly_select = utils.process_data(collected_keywords_data, '주별 급상승', '주별 급상승', select_list[1])
info_result_monthly_select = utils.process_data(collected_keywords_data, '월별 급상승', '월별 급상승', select_list[2]) 

info_result_weekly_continuous = utils.process_data(collected_keywords_data, '주별 지속상승', '주별 지속상승', rising_list[0])

info_result_monthly_continuous = utils.process_data(collected_keywords_data, '월별 지속상승', '월별 지속상승', rising_list[1])

info_result_monthly_pattern = utils.process_data(collected_keywords_data, '월별 규칙성', '월별 규칙성', month_rule_list)

info_result_final = pd.concat([info_result_daily_select,info_result_weekly_select, info_result_monthly_select,\
                               info_result_weekly_continuous, info_result_monthly_continuous,\
                                  info_result_monthly_pattern]).reset_index(drop=True)
info_result_final

AttributeError: module 'utils' has no attribute 'process_data'

# 구글/ 네이버 한꺼번에

### 구글 검색어 수집

In [18]:
import asyncio
import time
from pytrends.request import TrendReq
from concurrent.futures import ThreadPoolExecutor

# Google Trends에 대한 비동기 조회를 위한 함수

# 비동기 작업을 위한 ThreadPoolExecutor 인스턴스 생성
executor = ThreadPoolExecutor(max_workers=5)

async def fetch_rising_queries(keyword: str, max_retries: int = 5) -> dict:
    pytrends = TrendReq(hl='ko-KR', tz=540, retries=2)
    kw_list = [keyword]

    for attempt in range(max_retries):
        try:
            loop = asyncio.get_running_loop()
            # Google Trends payload를 비동기적으로 구축
            await loop.run_in_executor(executor, lambda: pytrends.build_payload(kw_list, timeframe='today 1-m', geo='KR', gprop=''))
            # 관련 검색어 조회를 비동기적으로 실행
            result = await loop.run_in_executor(executor, pytrends.related_queries)
            rising_queries = result[keyword]['rising']
            if rising_queries is not None:
                return {keyword: list(rising_queries['query'])}
            return {keyword: []}
        except Exception as e:
            print(f"Error fetching data for {keyword}: {e}")
            # 지수 백오프를 사용한 재시도 대기
            await asyncio.sleep(attempt)
    return {keyword: []}  # 최대 재시도 후 빈 결과 반환

async def collect_rising_keywords(target_keywords: list) -> dict:
    # 주어진 키워드 리스트에 대해 비동기 조회 작업 생성
    tasks = [fetch_rising_queries(keyword) for keyword in target_keywords]
    # 모든 작업의 완료를 기다림
    results = await asyncio.gather(*tasks)
    aggregated_results = {}
    for result in results:
        aggregated_results.update(result)
    return aggregated_results

# 사용할 키워드 리스트 예시 (실제 사용시 info_result_final에서 추출)
target_keywords = list(set(info_result_final['연관키워드']))

# 비동기 함수 실행하여 결과 수집
rising_keywords_results = asyncio.run(collect_rising_keywords(target_keywords))
google_trend = rising_keywords_results

Error fetching data for 상장컨설팅: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=540&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22%5Cuc0c1%5Cuc7a5%5Cucee8%5Cuc124%5Cud305%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))
Error fetching data for 연금보험추천: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=ko-KR&tz=540&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22%5Cuc5f0%5Cuae08%5Cubcf4%5Cud5d8%5Cucd94%5Cucc9c%22%2C+%22time%22%3A+%22today+1-m%22%2C+%22geo%22%3A+%22KR%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))
Error fetching data for 유품: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api

##### 뉴스링크,제목 수집 (네이버)

In [19]:
import asyncio
import aiohttp
from difflib import SequenceMatcher
import nest_asyncio

nest_asyncio.apply()

def clean_text(text):
    """
    HTML 태그 제거 및 특수 문자 처리 함수
    """
    text = text.replace('<b>', ' ').replace('</b>', ' ')
    text = text.replace('&quot;', '"').replace('&apos;', '\'')
    text = text.replace('amp;', '').replace('&lt;', '<').replace('&gt;', '>')
    return text


async def fetch_news(session, search_term, headers, params, attempts=10):
    """
    네이버 API를 통해 뉴스 데이터를 비동기적으로 가져오는 함수
    """
    backoff = 1
    for attempt in range(attempts):
        try:
            async with session.get("https://openapi.naver.com/v1/search/news.json", headers=headers, params=params) as response:
                if response.status == 200:
                    data = await response.json()
                    return data['items']
                else:
                    raise Exception(f"Error fetching data for {search_term}: {response.status}")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt < attempts - 1:
                await asyncio.sleep(3 * backoff)  # 실패 시 지수적 백오프 적용 -> 오래걸리면 지수말고 고정/선형
                backoff *= 2
            else:
                raise

async def news_result_async(names):
    """
    검색어별 뉴스 제목, 본문, 링크 추출 및 필터링하는 비동기 함수
    """
    client_id = "ByXmMvAqMIxyVUY_h17L"  # 네이버 API 클라이언트 ID
    client_secret = "2x7yByvNSN"  # 네이버 API 클라이언트 Secret
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }
    
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_news(session, name, headers, {"query": name, "display": 100, "sort": "date"}) for name in names]
        results = await asyncio.gather(*tasks)
        
        results_dict = {}
        for name, result in zip(names, results):
            titles_links = [(clean_text(item['title']), item['link']) for item in result[:10] if result]
            results_dict[name] = titles_links
            
    return results_dict

async def process_keywords_batch(batch):
    """
    키워드 배치 처리를 위한 함수
    """
    return await news_result_async(batch)

async def main(keywords):
    """
    비동기 실행을 위한 메인 함수, 키워드 리스트를 분할하여 처리
    """
    batch_size = 20 # 한 번에 처리할 키워드 수 # 20 : 160
    batches = [keywords[i:i + batch_size] for i in range(0, len(keywords), batch_size)]
    results = []

    for batch in batches:
        results.append(await process_keywords_batch(batch))
    
    # 결과 병합
    merged_results = {}
    for batch_result in results:
        merged_results.update(batch_result)

    return merged_results

# 실행
news_data = asyncio.run(main(target_keywords))

for keyword, data in news_data.items():
    print(f"Keyword: {keyword}")
    for title, link in data:
        print(f"Title: {title}, Link: {link}")


Attempt 1 failed: Error fetching data for 3D프린터구매: 429
Attempt 1 failed: Error fetching data for 목업: 429
Attempt 1 failed: Error fetching data for 미국주식증권사: 429
Attempt 1 failed: Error fetching data for 오늘순금가격: 429
Attempt 1 failed: Error fetching data for 코인거래소: 429
Attempt 1 failed: Error fetching data for 자동차보험: 429
Attempt 1 failed: Error fetching data for 애플주가: 429
Attempt 1 failed: Error fetching data for 코인상담: 429
Attempt 1 failed: Error fetching data for 등기: 429
Attempt 1 failed: Error fetching data for 건강보험비교사이트: 429
Attempt 1 failed: Error fetching data for 3대질환보험: 429
Attempt 1 failed: Error fetching data for IPO컨설팅: 429
Attempt 1 failed: Error fetching data for 자동차보험료견적비교: 429
Attempt 1 failed: Error fetching data for 상속전문변호사: 429
Attempt 1 failed: Error fetching data for CU-NI: 429
Attempt 1 failed: Error fetching data for 버그클라우드: 429
Attempt 1 failed: Error fetching data for 주식장: 429
Attempt 1 failed: Error fetching data for 증여세상담: 429
Attempt 1 failed: Error fetching data

In [20]:
#######################################
# 뉴스링크,제목,연관검색어 데이터프레임 생성
#######################################


name_list = list(news_data.keys())  
# DataFrame 초기화
news_df = pd.DataFrame()

# 모든 키워드에 대해 처리
for keyword in name_list:
    # 뉴스 항목이 있는 경우 데이터 추가
    for news_item in news_data[keyword]:
        news_row = [keyword, news_item[0], news_item[1]]  # 연관키워드, 뉴스제목, 뉴스링크
        news_df = pd.concat([news_df, pd.DataFrame([news_row])], ignore_index=True)

    # 뉴스 항목 수가 10개에 미치지 못하면 나머지를 빈 행으로 채움
    for _ in range(10 - len(news_data[keyword])):
        empty_row = [keyword, None, None]  # 연관키워드, 빈 뉴스제목, 빈 뉴스링크
        news_df = pd.concat([news_df, pd.DataFrame([empty_row])], ignore_index=True)

# 칼럼 이름 설정
news_df.columns = ['연관검색어', '뉴스제목', '뉴스링크']
news_df.head()


,연관검색어,뉴스제목,뉴스링크
0,종이신문,"무림, 국내 최초 폐의류 활용 자원순환형 포장용지 개발",http://www.beyondpost.co.kr/view.php?ud=202403...
1,종이신문,"무림, 국내 최초 폐의류 활용 업사이클링 포장용지 개발",https://www.getnews.co.kr/news/articleView.htm...
2,종이신문,"무림, 헌 옷으로 만든 자원순환형 포장용지 국내 최초 출시",http://www.ujeil.com/news/articleView.html?idx...
3,종이신문,"허위 사실 보도한 기자, 명예훼손죄로 법정行",http://www.newstnt.com/news/articleView.html?i...
4,종이신문,"국내 최초 버려진 옷 활용 자원순환형 종이 , 무림 국내 최초 개발",http://www.thebigdata.co.kr/view.php?ud=202403...


활동성 수집 및 삽입

In [21]:
with open('target_keywords.txt', 'w') as file:
    for keyword in target_keywords:
        file.write("%s\n" % keyword)

models/naver/blog.py 실행 (활동성 데이터 수집하는 파이썬 함수파일)


In [22]:
%run models/naver/blog.py

Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A2%85%EC%9D%B4%EC%8B%A0%EB%AC%B8&display=100&start=1&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A2%85%EC%9D%B4%EC%8B%A0%EB%AC%B8&display=100&start=501&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A2%85%EC%9D%B4%EC%8B%A0%EB%AC%B8&display=100&start=101&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A2%85%EC%9D%B4%EC%8B%A0%EB%AC%B8&display=100&start=401&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A2%85%EC%9D%B4%EC%8B%A0%EB%AC%B8&display=100&start=201&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A2%85%EC%9D%B4%EC%8B%A0%EB%AC%B8&display=100&start=701&sort=date
Successfully fetched data for URL: https://openapi.naver.com/v1/search/blog?query=%EC%A2%85%EC%9

In [128]:
keyword_activity_rates = pd.read_csv('keyword_activity_rates.csv')
keyword_activity_rates.columns = ['연관검색어', '활동성']

# '활동성' 열의 데이터를 백분율 형태의 문자열로 변환
keyword_activity_rates['활동성'] = keyword_activity_rates['활동성'].apply(lambda x: f"{x}%")
# news_df와 keyword_activity_rates를 '연관검색어' 열을 기준으로 병합
keyword_activity_rates = keyword_activity_rates.drop_duplicates(subset=['연관검색어'])
merged_keyword_activity_rates = pd.merge(news_df, keyword_activity_rates, on='연관검색어', how='left')

merged_keyword_activity_rates

,연관검색어,뉴스제목,뉴스링크,활동성
0,종이신문,"무림, 국내 최초 폐의류 활용 자원순환형 포장용지 개발",http://www.beyondpost.co.kr/view.php?ud=202403...,2.1%
1,종이신문,"무림, 국내 최초 폐의류 활용 업사이클링 포장용지 개발",https://www.getnews.co.kr/news/articleView.htm...,2.1%
2,종이신문,"무림, 헌 옷으로 만든 자원순환형 포장용지 국내 최초 출시",http://www.ujeil.com/news/articleView.html?idx...,2.1%
3,종이신문,"허위 사실 보도한 기자, 명예훼손죄로 법정行",http://www.newstnt.com/news/articleView.html?i...,2.1%
4,종이신문,"국내 최초 버려진 옷 활용 자원순환형 종이 , 무림 국내 최초 개발",http://www.thebigdata.co.kr/view.php?ud=202403...,2.1%
...,...,...,...,...
4885,실손보험가입,백내장 환자 알선 브로커 통해 연매출 230억,https://n.news.naver.com/mnews/article/009/000...,0.0%
4886,실손보험가입,"신안군,군민안전 보험 확대 시행..상해의료비도 추가",https://www.viva100.com/main/view.php?key=2024...,0.0%
4887,실손보험가입,2024년 신안군 군민 안전 보험 확대 시행,http://www.bzeronews.com/news/articleView.html...,0.0%
4888,실손보험가입,"신안군, 군민 안전 보험 확대 시행",https://www.psnews.co.kr/news/articleView.html...,0.0%


네이버 검색어 merge

In [129]:
collected_keywords_dat_copy.rename(columns={'연관키워드': '연관검색어'}, inplace=True)
info_result_final.rename(columns={'연관키워드': '연관검색어'}, inplace=True)
# collected_keywords_dat_copy에서 '연관키워드'와 '검색어'를 기준으로 중복 제거
collected_keywords_dat_copy = collected_keywords_dat_copy.drop_duplicates(subset=['연관검색어'], keep='first')
# 이제 merged_keyword_activity_rates와 결합
final_merged_df = pd.merge(merged_keyword_activity_rates, collected_keywords_dat_copy[['연관검색어', '검색어']], on='연관검색어', how='left')



In [130]:
final_merged_df_copy = final_merged_df.copy()

# 구글검색어 컬럼을 초기화합니다.
final_merged_df_copy['구글검색어'] = None

# 이후의 모든 작업은 final_merged_df_copy에 대해 수행합니다.
i = 0
for keyword, queries in rising_keywords_results.items():
    filled_queries = queries[:10] + [None] * (10 - len(queries[:10]))
    for query in filled_queries:
        if i < len(final_merged_df_copy):
            final_merged_df_copy.at[i, '구글검색어'] = query
            i += 1
        else:
            break


# final_merged_df의 '검색어' 컬럼에서 각 10번째 검색어를 추출합니다.
keyword_list_per_10 = final_merged_df_copy['검색어'].tolist()[::10]


 
# collected_keywords_dat_copy에서 각 검색어별 상위 10개 연관검색어를 가져옵니다.
# 여기서는 각 검색어별로 가장 높은 월간검색수를 가진 상위 10개를 선정합니다.
top_keywords_by_search = collected_keywords_dat_copy.groupby('검색어').apply(
    lambda x: x.nlargest(10, '월간검색수_합계')
).reset_index(drop=True)



# 새로운 DataFrame을 초기화합니다. 이 DataFrame에는 각 검색어별 상위 10개 연관검색어가 포함됩니다.
new_rows_for_final_df = []


for keyword in keyword_list_per_10:
    # 특정 키워드에 대한 상위 10개 연관 검색어 추출
    top_queries_for_keyword = top_keywords_by_search[top_keywords_by_search['검색어'] == keyword].head(10)
    
    # 추출된 연관 검색어를 결과 리스트에 추가
    num_rows_added = 0  # 추가된 연관 검색어의 수를 추적
    for _, row in top_queries_for_keyword.iterrows():
        new_rows_for_final_df.append(row['연관검색어'])
        num_rows_added += 1
    
    # 10개 미만인 경우 나머지를 None으로 채우기
    for _ in range(10 - num_rows_added):
        new_rows_for_final_df.append(None)


# new_rows_for_final_df의 길이를 확인하고 final_merged_df의 '네이버검색어' 컬럼에 값을 할당합니다.
# 주의: new_rows_for_final_df의 길이가 final_merged_df의 행 수와 동일해야 합니다.
# 만약 길이가 다르다면, 길이가 맞도록 조정이 필요합니다.
if len(new_rows_for_final_df) == len(final_merged_df_copy):
    final_merged_df_copy['네이버검색어'] = new_rows_for_final_df
else:
    print("경고: '네이버검색어' 데이터의 길이가 final_merged_df와 다릅니다. 데이터 확인이 필요합니다.")

# 최종 DataFrame 확인
#final_merged_df_copy

C:\Users\chohy\AppData\Local\Temp\ipykernel_5716\1304442105.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_keywords_by_search = collected_keywords_dat_copy.groupby('검색어').apply(


# 형식 수정

In [261]:

#info_result_final = info_result_final.drop(columns=["일별 급상승", "주별 급상승", "주별 지속상승", "월별 급상승", "월별 지속상승", "월별 규칙성"])

final_merged_df_result = pd.merge(info_result_final, final_merged_df_copy, how='left', on='연관검색어')
today_date = datetime.now().strftime("%Y-%m-%d")

# '기준일자' 컬럼을 가장 앞에 추가
final_merged_df_result.insert(0, '기준일자', today_date)
# 컬럼명 변경: '중복검색어' -> '검색키워드', '월간검색수_합계' -> '검색량'

final_merged_df_result.rename(columns={'중복검색어': '검색키워드', '월간검색수_합계': '검색량'}, inplace=True)

final_merged_df_result = final_merged_df_result.drop(columns=["검색어"])


final_merged_df_result['상승월'] = None
# rising_month_list의 각 항목에 대해 반복 처리
for month_info in rising_month_list:
    months, keyword = month_info  # month_info는 각각의 월 목록과 키워드를 포함합니다.
    keyword_rows = final_merged_df_result[final_merged_df_result['연관검색어'] == keyword]  # 해당 키워드에 대한 행만 선택합니다.
    
    if not keyword_rows.empty:

        for i, month in enumerate(months):
            if i < len(keyword_rows):
                final_merged_df_result.loc[keyword_rows.index[i], '상승월'] = month
            else:
                break  # 월의 개수보다 더 많은 행에 대해서는 처리를 중단합니다.



# 형식맞추기 위한 info_result_final 순서 정렬
info_result_af_copy=pd.DataFrame()
a = final_merged_df_result.query("`유형` == '일별 급상승'")
b = final_merged_df_result.query("`유형` == '주별 급상승' or `유형` == '주별 지속상승'")
c = final_merged_df_result.query("`유형` == '월별 급상승' or `유형` == '월별 지속상승' or `유형` == '월별 규칙성'")
a_sort=a.sort_values(by=['연관검색어', '유형'], ascending=[True, True])
b_sort = b.sort_values(by=['연관검색어', '유형'], ascending=[True, True])
c_sort = c.sort_values(by=['연관검색어', '유형'], ascending=[True, True])
info_result_af_copy=pd.concat([a_sort,b_sort,c_sort])

# 형식을 위한 이름 변경
new_column_order = ['기준일자', '유형', '연관검색어', '검색키워드', '검색량', '지표', '뉴스제목', '뉴스링크', '활동성', '구글검색어', '네이버검색어', '상승월']
info_result_af_copy_reordered = info_result_af_copy[new_column_order]

# 혹시나 모를 예외처리(형식에 어긋나는 것을 예방)
info_result_af_copy_reordered['뉴스제목'] = info_result_af_copy_reordered['뉴스제목'].str.replace("|", "")
info_result_af_copy_reordered['연관검색어'] = info_result_af_copy_reordered['연관검색어'].str.replace("|", "")
info_result_af_copy_reordered['네이버검색어'] = info_result_af_copy_reordered['네이버검색어'].str.replace("|", "")
info_result_af_copy_reordered['구글검색어'] = info_result_af_copy_reordered['구글검색어'].str.replace("|", "")
info_result_af_copy_reordered['뉴스제목'] = info_result_af_copy_reordered['뉴스제목'].str.replace("–", "-")


In [262]:
# 유형 순서 정렬
info_result_af_copy_reordered_modified = info_result_af_copy_reordered.copy()


# 인덱스 재설정
info_result_af_copy_reordered_modified.reset_index(drop=True, inplace=True)

sort_order = {
    "일별 급상승": 1,
    "주별 급상승": 2,
    "주별 지속상승": 3,
    "월별 급상승": 4,
    "월별 지속상승": 5,
    "월별 규칙성" : 6
}

# 유형 컬럼에 대한 정렬 순서를 적용하기 위해 임시 컬럼 추가
info_result_af_copy_reordered_modified['sort_key'] = info_result_af_copy_reordered_modified['유형'].map(sort_order)

# 임시 컬럼을 기준으로 정렬
info_result_af_copy_reordered_modified = info_result_af_copy_reordered_modified.sort_values(by=['sort_key', '연관검색어'], ascending=[True, True])

# 임시 컬럼 삭제
info_result_af_copy_reordered_modified.drop('sort_key', axis=1, inplace=True)

info_result_af_copy_reordered_modified.reset_index(drop=True, inplace=True)

info_result_af_copy_reordered_modified[info_result_af_copy_reordered_modified['유형']=='월별 규칙성']

,기준일자,유형,연관검색어,검색키워드,검색량,지표,뉴스제목,뉴스링크,활동성,구글검색어,네이버검색어,상승월
8210,2024-03-13,월별 규칙성,14K금시세,금값,50470.0,93.75%,오늘의 금값시세(3월 13일자) '2024년 사우디아라비아 금가격 전망',https://www.gukjenews.com/news/articleView.htm...,0.0%,14k 금 시세,금시세,None
8211,2024-03-13,월별 규칙성,14K금시세,금값,50470.0,93.75%,3월 12일 금시세 (금값)는?,https://kizmom.hankyung.com/news/view.html?aid...,0.0%,None,금값시세,None
8212,2024-03-13,월별 규칙성,14K금시세,금값,50470.0,93.75%,금시세 (금값) 12일 0.154%↓,https://www.bntnews.co.kr/article/view/bnt2024...,0.0%,None,금값,None
8213,2024-03-13,월별 규칙성,14K금시세,금값,50470.0,93.75%,"[오늘 금시세 ]오늘 금값 상승세↑… 원 달러 환율 1,312원",http://www.namdonews.com/news/articleView.html...,0.0%,None,오늘의금시세,None
8214,2024-03-13,월별 규칙성,14K금시세,금값,50470.0,93.75%,오늘의 금값시세(3월 12일자) '2024년 러시아 금가격 전망',https://www.gukjenews.com/news/articleView.htm...,0.0%,None,오늘금시세,None
...,...,...,...,...,...,...,...,...,...,...,...,...
8635,2024-03-13,월별 규칙성,해외선물,"ETF,WTI,급등주,달러환율,테마주",16120.0,96.11%,"트라이비, 故 신사동호랭이 사망 아픔 딛고 활동 성료 ""위로되길""",https://n.news.naver.com/mnews/article/076/000...,25.8%,None,금양주가,None
8636,2024-03-13,월별 규칙성,해외선물,"ETF,WTI,급등주,달러환율,테마주",16120.0,96.11%,"나 영입해 놓고 나가네...'지지파' 케인, 투헬 사임 발표에 '충격'",https://sports.news.naver.com/news.nhn?oid=311...,25.8%,None,삼성SDI주가,None
8637,2024-03-13,월별 규칙성,해외선물,"ETF,WTI,급등주,달러환율,테마주",16120.0,96.11%,"[굿모닝브리핑] ""리뷰 체험단 하세요"" 신종 보이스피싱 수법 활개",https://n.news.naver.com/mnews/article/052/000...,25.8%,None,HMM주가,None
8638,2024-03-13,월별 규칙성,해외선물,"ETF,WTI,급등주,달러환율,테마주",16120.0,96.11%,거래소 '밸류업TF' 오늘 정식 출범…5월 가이드라인 공개 위해 '박차',https://n.news.naver.com/mnews/article/421/000...,25.8%,None,실시간나스닥선물지수,None


In [290]:
# 형식에 맞춰서 띄어쓰기 변경
# '유형' 컬럼의 값을 바꾸기 위한 딕셔너리 정의
replace_values = {
    '일별 급상승': '일별급상승',
    '주별 급상승': '주별급상승',
    '주별 지속상승': '주별지속상승',
    '월별 급상승': '월별급상승',
    '월별 지속상승': '월별지속상승',
    '월별 규칙성': '월별규칙성'
}

# '유형' 컬럼 내의 값을 바꾸기
graph_result['유형'] = graph_result['유형'].replace(replace_values)

### nan값 제거

In [291]:
excluding_monthly_rise_df = graph_result[graph_result['유형'] != '월별급상승']
a = list(graph_result[pd.isna(graph_result['검색량'])]['연관검색어'])
a_unique = list(set(a))

filtered_graph_result = graph_result[~graph_result['연관검색어'].isin(a_unique)]


filtered_info_result_af_copy_reordered_modified = info_result_af_copy_reordered_modified[~info_result_af_copy_reordered_modified['연관검색어'].isin(a_unique)]

# info_result_af_out csv 생성

In [293]:
column_names = '|||'.join(filtered_info_result_af_copy_reordered_modified[:-1]) + '|||'+'상승월|||'


# 각 행을 '|||'로 구분된 문자열로 변환하고 새로운 DataFrame의 한 컬럼으로 저장
df_string = filtered_info_result_af_copy_reordered_modified.apply(lambda x: '|||'.join(x.fillna(' ').astype(str))+'|||', axis=1)

new_df = pd.DataFrame(df_string, columns=[column_names])

today_date = datetime.now().strftime("%y%m%d")

base_save_path = "./data/result_out"

# 오늘 날짜를 포함한 최종 저장 경로 생성
final_save_path = os.path.join(base_save_path, today_date)

# 최종 저장 경로가 없는 경우 생성
if not os.path.exists(final_save_path):
    os.makedirs(final_save_path)

# 파일 저장 경로 설정
file_save_path = os.path.join(final_save_path, f"info_result_af_out_{today_date}.csv")

# DataFrame을 CSV 파일로 저장
new_df.to_csv(file_save_path, index=False, encoding='utf-8-sig')
print(f"File saved at: {file_save_path}")

File saved at: ./data/result_out\240313\info_result_af_out_240313.csv


# graph_result_out csv 생성

In [292]:
column_names = '|||'.join(filtered_graph_result.columns[:-1]) +'|||검색량|||'


# 각 행을 '|||'로 구분된 문자열로 변환하고 새로운 DataFrame의 한 컬럼으로 저장

df_string = filtered_graph_result.apply(lambda x: '|||'.join(x.astype(str)) + '|||', axis=1)
new_df = pd.DataFrame(df_string, columns=[column_names])

today_date = datetime.now().strftime("%y%m%d")

base_save_path = "./data/result_out"

# 오늘 날짜를 포함한 최종 저장 경로 생성
final_save_path = os.path.join(base_save_path, today_date)

# 최종 저장 경로가 없는 경우 생성
if not os.path.exists(final_save_path):
    os.makedirs(final_save_path)

# 파일 저장 경로 설정
file_save_path = os.path.join(final_save_path, f"graph_result_out_{today_date}.csv")

# DataFrame을 CSV 파일로 저장
new_df.to_csv(file_save_path, index=False, encoding='utf-8-sig')
print(f"File saved at: {file_save_path}")

File saved at: ./data/result_out\240313\graph_result_out_240313.csv
